<a href="https://colab.research.google.com/github/vfrantc/trans_experiments/blob/main/processing_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi 

Mon Jun  6 15:19:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tqdm
!pip install thop
!pip install kornia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 512 kB 5.1 MB/s 


In [3]:
import os 
import cv2
from glob import glob 
from tqdm.notebook import tqdm 

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# Run Transweather

In [5]:
!git clone https://github.com/vfrantc/TransMod.git
%cd TransMod
!mkdir -p data/test
!cp /content/drive/MyDrive/deweather/input.zip .
!unzip input.zip

Cloning into 'TransMod'...
remote: Enumerating objects: 423, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 423 (delta 103), reused 84 (delta 48), pack-reused 264
Receiving objects: 100% (423/423), 11.13 MiB | 23.02 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/content/TransMod
Archive:  input.zip
   creating: input/
  inflating: input/get_list.py.ipynb  
  inflating: input/.DS_Store         
  inflating: __MACOSX/input/._.DS_Store  
  inflating: input/natural.txt       
   creating: input/input/
   creating: input/.ipynb_checkpoints/
  inflating: input/input/348.png     
  inflating: input/input/412.png     
  inflating: input/input/374.png     
  inflating: input/input/360.png     
  inflating: input/input/406.png     
  inflating: input/input/638.png     
  inflating: input/input/176.png     
  inflating: input/input/610.png     
  inflating: input/input/604.png     
  inflating: input/input/162.png     

In [6]:
for fname in tqdm(glob('input/input/*.png')):
  image = cv2.imread(fname)
  h, w = image.shape[:2]
  if h > w:
      image = cv2.resize(image, (int(w * 640 / h), 640))
  else:
      image = cv2.resize(image, (640, int(h * 640 / w)))

  cv2.imwrite(fname, image)

  0%|          | 0/497 [00:00<?, ?it/s]

In [7]:
!cp -r input data/test/
!cp input/natural.txt data/test/input.txt
!mkdir -p data/test/gt
!cp -r input/input data/test/gt/gt

In [8]:
!rm -rf Transweather
!cp /content/drive/MyDrive/derain/Transweather_orig.zip .
!unzip Transweather_orig.zip
!python testme.py -exp_name=./Transweather

Archive:  Transweather_orig.zip
   creating: Transweather/
  inflating: Transweather/latest     
  inflating: Transweather/best       
Seed:	19
cuda
--- Testing starts! ---
348.png
412.png
374.png
360.png
406.png
638.png
176.png
610.png
604.png
162.png
002.png
016.png
228.png
566.png
200.png
214.png
572.png
599.png
215.png
573.png
567.png
201.png
229.png
017.png
003.png
605.png
163.png
177.png
611.png
639.png
407.png
413.png
375.png
349.png
363.png
377.png
411.png
149.png
161.png
607.png
613.png
175.png
015.png
001.png
559.png
571.png
203.png
565.png
202.png
564.png
570.png
216.png
558.png
014.png
028.png
612.png
174.png
160.png
606.png
389.png
376.png
410.png
404.png
362.png
366.png
400.png
414.png
372.png
399.png
602.png
170.png
616.png
158.png
010.png
004.png
038.png
212.png
574.png
560.png
206.png
561.png
207.png
213.png
575.png
039.png
005.png
011.png
159.png
171.png
617.png
603.png
165.png
398.png
415.png
367.png
401.png
371.png
365.png
359.png
615.png
167.png
601.png
629.png
198

In [9]:
!mkdir -p /content/data
!cp -r data/test/input/input /content/data
!cp -r ./results/natural/Transweather /content/data/deweather 

In [10]:
%cd ..

/content


# Run retinexnet 

In [11]:
!git clone https://github.com/aasharma90/RetinexNet_PyTorch.git
%cd RetinexNet_PyTorch/

Cloning into 'RetinexNet_PyTorch'...
remote: Enumerating objects: 51, done.
remote: Total 51 (delta 0), reused 0 (delta 0), pack-reused 51
Unpacking objects: 100% (51/51), done.
/content/RetinexNet_PyTorch


In [12]:
!mkdir -p /content/data/retinexnet
!python predict.py --data_dir /content/data/deweather --res_dir /content/data/retinexnet

Number of evaluation images: 497
Decom   : Model restore success!
Relight : Model restore success!
Processing  001.png
Processing  002.png
Processing  003.png
Processing  004.png
Processing  005.png
Processing  006.png
Processing  007.png
Processing  008.png
Processing  009.png
Processing  010.png
Processing  011.png
Processing  012.png
Processing  013.png
Processing  014.png
Processing  015.png
Processing  016.png
Processing  017.png
Processing  020.png
Processing  021.png
Processing  022.png
Processing  024.png
Processing  026.png
Processing  027.png
Processing  028.png
Processing  030.png
Processing  034.png
Processing  036.png
Processing  038.png
Processing  039.png
Processing  041.png
Processing  042.png
Processing  043.png
Processing  044.png
Processing  045.png
Processing  046.png
Processing  047.png
Processing  048.png
Processing  049.png
Processing  050.png
Processing  051.png
Processing  052.png
Processing  053.png
Processing  054.png
Processing  055.png
Processing  056.png
P

# Run median + sharpening

In [13]:
%cd ..

/content


In [29]:
import cv2
import os
from glob import glob 
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt 
import numpy as np

In [23]:
for fname in glob('/content/data/retinexnet/*.jpg'):
  image = cv2.imread(fname)
  height, width = image.shape[:2]
  image = image[:, (width//2):, :]
  cv2.imwrite(fname, image)

In [17]:
#!cp -r TransMod/data/test/input/input /drive/data

In [28]:
!mkdir -p /content/data/postprocessed

In [30]:
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    # https://en.wikipedia.org/wiki/Unsharp_masking
  
    blurred = cv2.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [31]:
out_dir = '/content/data/postprocessed'
for fname in tqdm(glob('/content/data/retinexnet/*.jpg')):
  imname = os.path.basename(fname)
  image = cv2.imread(fname)
  image = cv2.medianBlur(image, 3)  
  image = unsharp_mask(image, sigma=2.0)
  cv2.imwrite(os.path.join(out_dir, imname), image)

  0%|          | 0/497 [00:00<?, ?it/s]

In [37]:
!mkdir /content/data/weighted

In [41]:
for fname in tqdm(glob('/content/data/deweather/*.png')):
  fname = os.path.basename(fname)[:-4]
  fn_in_orig = os.path.join('/content/data/deweather/', fname + '.png')
  fn_in_color = os.path.join('/content/data/retinexnet/', fname + '.jpg')
  fn_out = os.path.join('/content/data/weighted', fname + '.jpg')

  im_orig = cv2.imread(fn_in_orig)
  im_color = cv2.imread(fn_in_color)
  im_out = (0.8 * im_orig + 0.2 * im_color).astype(np.uint8)
  im_out = cv2.medianBlur(im_out, 3)  
  im_out = unsharp_mask(im_out, sigma=2.0)
  cv2.imwrite(fn_out, im_out)

  0%|          | 0/497 [00:00<?, ?it/s]

# Join for the presentation

In [42]:
!zip -r outout2.zip /content/data

updating: content/data/ (stored 0%)
updating: content/data/weighted/ (stored 0%)
updating: content/data/weighted/015.jpg (deflated 0%)
updating: content/data/weighted/599.jpg (deflated 0%)
updating: content/data/weighted/407.jpg (deflated 0%)
updating: content/data/weighted/606.jpg (deflated 0%)
updating: content/data/weighted/588.jpg (deflated 0%)
updating: content/data/weighted/295.jpg (deflated 0%)
updating: content/data/weighted/562.jpg (deflated 0%)
updating: content/data/weighted/158.jpg (deflated 0%)
updating: content/data/weighted/612.jpg (deflated 0%)
updating: content/data/weighted/179.jpg (deflated 0%)
updating: content/data/weighted/209.jpg (deflated 0%)
updating: content/data/weighted/642.jpg (deflated 0%)
updating: content/data/weighted/633.jpg (deflated 0%)
updating: content/data/weighted/494.jpg (deflated 0%)
updating: content/data/weighted/078.jpg (deflated 0%)
updating: content/data/weighted/644.jpg (deflated 0%)
updating: content/data/weighted/564.jpg (deflated 0%)
u

In [43]:
!cp outout2.zip /content/drive/MyDrive/